In [1]:
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--lang=en_US')
options.add_argument('--disable-notifications')##!!!important for ignore download pop-up
options.add_argument('window-size=1920,1080');##Change reslution if element can't interact
options.add_argument('--user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"')
##change UA in above code if CCB not loading correctly
#options.add_argument('--disable-dev-shm-usage')

In [3]:
desired_capabilities = DesiredCapabilities.CHROME
desired_capabilities["pageLoadStrategy"] = 'none'
options.add_experimental_option('prefs', {
"download.default_directory": "pdfs/CCB", #Change default directory for downloads
"download.prompt_for_download": False, #To auto download the file
"download.directory_upgrade": True,
"plugins.always_open_pdf_externally": True
 #It will not show PDF directly in chrome
})

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
#driver.set_window_size(1920, 1080)
wait = WebDriverWait(driver, 60)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/home/jovyan/.wdm/drivers/chromedriver/linux64/84.0.4147.30/chromedriver] found in cache


In [5]:
def openPage(driver):
    link = "http://finance.ccb.com/cn/finance/product.html"
    loading_table = "//*[@id='list1']/table/tbody/tr/td[1]/div/div/a"
    fund_tab = "/html/body/div[9]/div[1]/a[2]"
    status = "//*[@id='status']"
    all_status = "/html/body/div[7]/div/div[2]/ul/div[2]/ul/li[1]/a"
    driver.get(link)
    wait.until(EC.element_to_be_clickable((By.XPATH, loading_table)))
    driver.refresh()
    wait.until(EC.element_to_be_clickable((By.XPATH, loading_table)))
    wait.until(EC.element_to_be_clickable((By.XPATH, fund_tab))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, status))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, all_status))).click()
    time.sleep(2)

In [6]:
def locatePage(driver):
    page = "//*[@id='pageNum']"
    global all_pages
    global current_page
    wait.until(EC.element_to_be_clickable((By.XPATH, page)))
    pageN = driver.find_element_by_xpath(page).text
    all_pages = int(re.match("(\d).(\d.)",pageN).group(2))
    current_page = int(re.match("(\d).(\d.)", pageN).group(1))
    #print(current_page, '/' , all_pages)

In [7]:
def switchPage(driver):
    next_page = "//*[@id='next']"
    wait.until(EC.element_to_be_clickable((By.XPATH, next_page))).click()
    time.sleep(3)

In [8]:
def downloadCondition(current_page, all_pages):
    while current_page <= all_pages:
        print('Page: ', current_page, '/', all_pages)
        downloadPdf(driver)
        current_page = current_page +1
        if current_page > all_pages:
            break
        switchPage(driver)

In [9]:
def switchCity(driver):
    sale_city = "//*[@id='saleCitys']/li/a"
    sale_city_selector = "/html/body/div[7]/div/div[2]/ul/div[1]/div"
    city_switch = driver.find_elements_by_xpath(sale_city)
    for city in city_switch:
        downloadCondition(current_page, all_pages)
        wait.until(EC.element_to_be_clickable((By.XPATH, sale_city_selector))).click()
        print('now switch to ',city.get_attribute('textContent'))
        city.click()
        time.sleep(3)

In [10]:
def downloadPdf(driver):
    table = "//*[@id='list6']/table/tbody/tr/td[1]/div/div/a"
    product_info = "/html/body/div[8]/div[3]/div[1]/ul/li[2]/a[2]"
    download_link = "//*[@id='instructionUrl']/p/a"

    pdfs = driver.find_elements_by_xpath(table)
    for pdf in pdfs:
        if pdf.text not in passed_pdf:
            ##pdf.click()
            print('downloading', pdf.text)
            passed_pdf.insert(0,pdf.text)
            ##time.sleep(5)
            ##current_window = driver.window_handles
            #print(driver.title)
            #print(driver.current_window_handle)
            ##driver.switch_to.window(current_window[(len(current_window)-1)])
            ##wait.until(EC.element_to_be_clickable((By.XPATH, product_info))).click()
            ##wait.until(EC.element_to_be_clickable((By.XPATH, download_link))).click()
            ##time.sleep(3)
            #print(driver.title)
            #print(len(current_window))
            ##driver.close()
            ##driver.switch_to.window(current_window[0])
            #while len(current_window) > 1:
            #    driver.switch_to.window(current_window[(len(current_window)-1)])
            #    driver.close()
            #print(len(current_window))
            
        else:
            continue
    
    pdfs.clear()

In [ ]:
if __name__ == '__main__':
    openPage(driver)
    locatePage(driver)
    passed_pdf = pd.read_excel(r'../historical_fund/CCB.xlsx', sheet_name='ALL', index_col=None, header=None)[0].values.tolist()
    switchCity(driver)
    driver.quit()

Page:  1 / 20
downloading 代销建信理财诚益定增主题固收类677...
downloading 代销建信理财龙鑫固收类380天封闭第...
downloading 代销建信理财私行专享鹏鑫121天封闭...
downloading 代销建信理财龙鑫固收类552天封闭第...
downloading 代销建信理财龙鑫固收类420天封闭第...
downloading 代销建信理财私行专享福星367天封闭...
downloading 代销建信理财睿吉稳健养老固收类3年封...
downloading 代销建信理财睿鑫固收类1年封闭第42...
downloading 代销建信理财睿鑫最低持有60天开放第...
downloading 代销建信理财开鑫最低持有90天第1期
downloading 代销建信理财嘉鑫(最低持有180天)...
downloading 代销建信理财睿福稳健FOF(最低持1...
Page:  2 / 20
downloading 代销私行专享建信理财福星1个月定开第...
downloading 代销建信理财嘉鑫(7天)定开第1期
downloading 代销建信理财睿鑫(1个月)定开第1期
downloading 代销建信理财建信宝2号按日开放理财产...
downloading 代销建信理财建信宝1号按日开放理财产...
downloading 代销建信理财大湾区指数理财-最低持1...
downloading 建信理财龙宝(按日)净值型理财产品
downloading 代销建信理财龙鑫固收类12个月定开式...
downloading 代销建信理财龙鑫固收12个月定开式第...
downloading 代销建信理财睿鑫机构专享1个月定开式...
downloading 代销建信理财睿鑫1个月定开式（上旬）
downloading 代销建信理财睿鑫1个月定开式（下旬）
Page:  3 / 20
downloading 代销建信理财睿鑫6个月定开式第1期
downloading 代销建信理财嘉鑫(1个月)定开第1期
downloading 代销建信理财嘉鑫(3个月)定开第1期
downloading 代销建信理财睿智定开(3个月)第1期
downloading 代销建信理财睿智定开(12个月)第1...
downl

Page:  8 / 20
Page:  9 / 20
Page:  10 / 20
Page:  11 / 20
Page:  12 / 20
Page:  13 / 20
Page:  14 / 20
Page:  15 / 20
Page:  16 / 20
Page:  17 / 20
Page:  18 / 20
Page:  19 / 20
Page:  20 / 20
now switch to  甘肃省
Page:  1 / 20
Page:  2 / 20
Page:  3 / 20
Page:  4 / 20
Page:  5 / 20
Page:  6 / 20
Page:  7 / 20
Page:  8 / 20
Page:  9 / 20
Page:  10 / 20
Page:  11 / 20
Page:  12 / 20
Page:  13 / 20
Page:  14 / 20
Page:  15 / 20
Page:  16 / 20
Page:  17 / 20
Page:  18 / 20
Page:  19 / 20
Page:  20 / 20
now switch to  贵州省
Page:  1 / 20
Page:  2 / 20
Page:  3 / 20
Page:  4 / 20
Page:  5 / 20
Page:  6 / 20
Page:  7 / 20
Page:  8 / 20
Page:  9 / 20
Page:  10 / 20
Page:  11 / 20
downloading 代销建信理财私行专享鹏鑫1年封闭式2...
Page:  12 / 20
Page:  13 / 20
Page:  14 / 20
Page:  15 / 20
Page:  16 / 20
Page:  17 / 20
Page:  18 / 20
Page:  19 / 20
Page:  20 / 20
now switch to  海南省
Page:  1 / 20
Page:  2 / 20
Page:  3 / 20
Page:  4 / 20
Page:  5 / 20
Page:  6 / 20
Page:  7 / 20
Page:  8 / 20
Page:  9 / 20
Page:  1

Page:  7 / 20
Page:  8 / 20
Page:  9 / 20
Page:  10 / 20
downloading 代销建信理财嘉鑫封闭1年第49期(深...
Page:  11 / 20
downloading 代销建信理财嘉鑫封闭1年第40期(深...


In [ ]:
driver.quit()

In [ ]:
#passed_pdf= ['']
dfh = pd.DataFrame(passed_pdf)
#dfh.to_excel('../historical_fund/CCB.xlsx', sheet_name='ALL', header=False, index=False)

In [ ]:
#passed_pdf = pd.read_excel(r'../historical_fund/CCB.xlsx', sheet_name='ALL', index_col=None, header=None)[0].values.tolist()